With 1-dimensional CNN, Classify with the given raisin data!

In [1]:
import pandas as pd

raisin = pd.read_csv("https://raw.githubusercontent.com/KnightChaser/ML-challenge/main/EX_challenging_problems/02_raisin_classification/Raisin_Dataset.csv")
raisin.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,Extent,Perimeter,Class
0,87524,442.246011,253.291155,0.819738,90546,0.758651,1184.040,Kecimen
1,75166,406.690687,243.032436,0.801805,78789,0.684130,1121.786,Kecimen
2,90856,442.267048,266.328318,0.798354,93717,0.637613,1208.575,Kecimen
3,45928,286.540559,208.760042,0.684989,47336,0.699599,844.162,Kecimen
4,79408,352.190770,290.827533,0.564011,81463,0.792772,1073.251,Kecimen


In [2]:
# Which type does the dataframe have?

print(pd.unique(raisin["Class"]))

['Kecimen' 'Besni']


In [3]:
print(raisin.isna())

      Area  MajorAxisLength  MinorAxisLength  Eccentricity  ConvexArea  \
0    False            False            False         False       False   
1    False            False            False         False       False   
2    False            False            False         False       False   
3    False            False            False         False       False   
4    False            False            False         False       False   
..     ...              ...              ...           ...         ...   
895  False            False            False         False       False   
896  False            False            False         False       False   
897  False            False            False         False       False   
898  False            False            False         False       False   
899  False            False            False         False       False   

     Extent  Perimeter  Class  
0     False      False  False  
1     False      False  False  
2     False    

In [4]:
raisin_input  = raisin[["Area", "MajorAxisLength", "MinorAxisLength", "Eccentricity", "ConvexArea", "Extent", "Perimeter"]].to_numpy()
raisin_target = raisin["Class"].to_numpy()

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

train_input, test_input, train_target, test_target = train_test_split(raisin_input, raisin_target, random_state = 0xCAFE, test_size = 0.2)

ss = StandardScaler()
ss.fit(train_input)

train_scaled = ss.transform(train_input)
test_scaled  = ss.transform(test_input)

In [6]:
print(train_scaled.shape)
print(test_scaled.shape)

(720, 7)
(180, 7)


In [7]:
import numpy as np

train_target = [1 if x == "Kecimen" else 0 for x in train_target]
test_target  = [1 if x == "Kecimen" else 0 for x in test_target]

print(train_target[:20])
print(test_target[:20])

train_target = np.array(train_target)
test_target  = np.array(test_target)

[0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0]


In [8]:
from tensorflow import keras

def create_CNN_model(additional_layers = None):

    model = keras.Sequential()

    if additional_layers:
        for more_layer in additional_layers:
            model.add(more_layer)

    model.add(keras.layers.Dense(units = 2))        # output will be 2 types

    return model

In [9]:
## Build 1DCNN & Train

from sklearn.utils import validation
from keras.optimizers import SGD

def training_and_testing_1DCNN_Classifcation_model(unit_size = 64, dropout_rate = 0.3, dense_units = 16):

    model = create_CNN_model([
        keras.layers.Conv1D(unit_size, kernel_size = 2,                             # Convolution (1D)
                                activation  = "relu",
                                padding     = "same",
                                input_shape = (train_scaled.shape[1], 1)),
        keras.layers.Dropout(dropout_rate),                                         # Pooling (1D)
        keras.layers.MaxPooling1D(pool_size = 2),
        keras.layers.Flatten(),                                                     # Flatten
        keras.layers.Dense(units = dense_units, activation = "relu"),               # ANN
    ])

    opt = SGD(learning_rate = 0.03)

    model.compile(loss = "sparse_categorical_crossentropy",            # <--- CLASSIFICATION
                metrics = "accuracy",
                optimizer = "adam")

    early_stopping_checkpoint = keras.callbacks.EarlyStopping(patience = 6, restore_best_weights = True, mode = "auto")
    checkpoint_cb = keras.callbacks.ModelCheckpoint('best-cnn-model.h5', save_best_only=True)

    history = model.fit(train_scaled, train_target,
                        epochs = 100,
                        verbose = False,
                        validation_data = (test_scaled, test_target),
                        callbacks = [early_stopping_checkpoint, checkpoint_cb])

    result = model.evaluate(test_scaled, test_target, verbose = False)
    final_accuracy = result[1]

    return final_accuracy

In [12]:
# change unit_size
for unit_size in [2 ** x for x in range(7)]:
    accuracy = 0
    print("Progress[", end = "")
    for _seq in range(1, 5 + 1):
        accuracy += training_and_testing_1DCNN_Classifcation_model(unit_size)
        print("X", end = "")
    print("] ", end = "")
    print(f"Unit size: {unit_size} ----> average accuracy: {accuracy / _seq}")

Progress[XXXXX] Unit size: 1 ----> average accuracy: 0.6122222125530243
Progress[XXXXX] Unit size: 2 ----> average accuracy: 0.45777777433395384
Progress[XXXXX] Unit size: 4 ----> average accuracy: 0.5299999952316284
Progress[XXXXX] Unit size: 8 ----> average accuracy: 0.6555555701255799
Progress[XXXXX] Unit size: 16 ----> average accuracy: 0.6433333396911621
Progress[XXXXX] Unit size: 32 ----> average accuracy: 0.6211111128330231
Progress[XXXXX] Unit size: 64 ----> average accuracy: 0.7166666746139526


In [17]:
# According to the previous experiment, It seems that
# unit_size 64 was the best. Apply the result.

# change dropout_rate
for dropout_rate in  [x * 0.05 for x in range(0, 8)]:
    accuracy = 0
    print("Progress[", end = "")
    for _seq in range(1, 5 + 1):
        accuracy += training_and_testing_1DCNN_Classifcation_model(unit_size = 64, dropout_rate = dropout_rate)
        print("X", end = "")
    print("] ", end = "")
    print(f"Dropout rate: {dropout_rate:02f} ----> average accuracy: {accuracy / _seq}")

Progress[XXXXX] Dropout rate: 0.000000 ----> average accuracy: 0.6677777767181396
Progress[XXXXX] Dropout rate: 0.050000 ----> average accuracy: 0.748888897895813
Progress[XXXXX] Dropout rate: 0.100000 ----> average accuracy: 0.7655555605888367
Progress[XXXXX] Dropout rate: 0.150000 ----> average accuracy: 0.5166666686534882
Progress[XXXXX] Dropout rate: 0.200000 ----> average accuracy: 0.5977777779102326
Progress[XXXXX] Dropout rate: 0.250000 ----> average accuracy: 0.6066666722297669
Progress[XXXXX] Dropout rate: 0.300000 ----> average accuracy: 0.7611111044883728
Progress[XXXXX] Dropout rate: 0.350000 ----> average accuracy: 0.6744444489479064


In [18]:
# According to the previous experiment, It seems that
# dropout rate 0.3 was the best. Apply the result

# change dense_units
for dense_units in  [2 ** x for x in range(7)]:
    accuracy = 0
    print("Progress[", end = "")
    for _seq in range(1, 5 + 1):
        accuracy += training_and_testing_1DCNN_Classifcation_model(unit_size = 64, dropout_rate = 0.3, dense_units = dense_units)
        print("X", end = "")
    print("] ", end = "")
    print(f"Dense units: {dense_units} ----> average accuracy: {accuracy / _seq}")

Progress[XXXXX] Dense units: 1 ----> average accuracy: 0.5355555534362793
Progress[XXXXX] Dense units: 2 ----> average accuracy: 0.5511111259460449
Progress[XXXXX] Dense units: 4 ----> average accuracy: 0.7666666746139527
Progress[XXXXX] Dense units: 8 ----> average accuracy: 0.7566666603088379
Progress[XXXXX] Dense units: 16 ----> average accuracy: 0.7133333325386048
Progress[XXXXX] Dense units: 32 ----> average accuracy: 0.7155555605888366
Progress[XXXXX] Dense units: 64 ----> average accuracy: 0.7866666674613952


According to the previous results,

We can estimate

- Unit size `32`
- Dropout rate `0.300`
- Dense unit quantity `64`

will make pretty decent performanance
(Due to the lack of computing resource, I couldn't take tests on every probabilities of adjustable parameters, so it's not highly likely the best hyperparameter combination.)